# Clean DataFrames for Main Analysis
This notebook creates the main DataFrames for your analysis: df_lines, df_product, df_sites, df_line_product_assignment, and df_shift_configuration.

In [1]:
# 1. Import Required Libraries
import pandas as pd
import numpy as np

In [ ]:
# 2. Load Data
# Load the main sales data
data = pd.read_excel('./generated_data/fy26_sales_cheikh.xlsx', sheet_name=0)
data_df = pd.DataFrame(data)
# Select relevant columns
data_df = data_df[["FY", "Période", "Article", "Libellé", "Ligne", "UVC", "UVP", "Type Produit", "Type de recette", "Type de matière", "Type d'emballage", "Site"]]

In [3]:
# 3. Create df_sites
df_sites = data_df[["Site"]].drop_duplicates().reset_index(drop=True)
code_sites = ["PA02", "PA04", "PA05", "PA06", "PA03"]
df_sites["code_site"] = code_sites
df_sites

,Site,code_site
0,Dole,PA02
1,Hoerdt,PA04
2,Rivoli,PA05
3,SLB,PA06
4,Vittel,PA03


In [4]:
# 4. Create df_lines
df_lines = data_df[["Ligne", "Site"]].drop_duplicates().reset_index(drop=True)
df_lines = df_lines[df_lines["Ligne"] != "(vide)"]
df_lines = df_lines.merge(df_sites, on="Site", how="left")
df_lines.rename(columns={"Ligne": "Line code", "Site": "Site name", "code_site": "site code"}, inplace=True)
df_lines

,Line code,Site name,site code
0,PA02F05,Dole,PA02
1,PA04F02,Hoerdt,PA04
2,PA02F01,Dole,PA02
3,PA02F02,Dole,PA02
4,PA02F06,Dole,PA02
5,PA05F02,Rivoli,PA05
6,PA02F04,Dole,PA02
7,PA05F01,Rivoli,PA05
8,PA02F03,Dole,PA02
9,PA04F01,Hoerdt,PA04


In [5]:
# 5. Create df_product
df_product = data_df[["Article", "Libellé", "Type Produit", "Type de recette", "Type de matière", "Type d'emballage"]].drop_duplicates().reset_index(drop=True)
df_product.rename(columns={"Article": "product code", "Libellé": "description"}, inplace=True)
df_product

,product code,description,Type Produit,Type de recette,Type de matière,Type d'emballage
0,300361,CARREFOUR BRIS LOTX2 460G12CT SG,Roulée,brisée,MGV,Etui
1,300362,CARREFOUR FEUIL X2 460G 12CT SG,Roulée,feuilletée,MGV,Etui
2,300566,PINGO PT MASSA FOLHADA 230G 10CT MB,Roulée,feuilletée,MGV,FP_TRANSP
3,300567,PINGO PT MASSA QUEBRADA 230G 10CT MB,Roulée,brisée,MGV,FP_TRANSP
4,300802,CARREFOUR IT PASTA SFOGLIA 230G 10CT SG,Roulée,feuilletée,MGV,FPI_TOB
...,...,...,...,...,...,...
871,302762,NETTO FEUIL X2 EPIPH 460G 12CT SG,Roulée,feuilletée,MGV,Etui
872,304850,LABEL QUALITE KIT GALETTE 580G 8CT,Roulée,feuilletée,Beurre,KIT FLOW PACK
873,601592,ITALPIZZA BASE PER PIZZA RETT 400G 8CT,Roulée,Pizza,MGV,Etui
874,305344,DANEROLLES LIDL SUPER CROI 340G12CT SG,Roulée,Croissant,MGV,Boite


In [6]:
# 6. Create article_line_uvc and default_line_df
article_line_uvc = data_df.groupby(["Article", "Ligne"], as_index=False)["UVC"].sum()
article_line_uvc.rename(columns={"UVC": "Total_UVC"}, inplace=True)
default_line_df = article_line_uvc.loc[article_line_uvc.groupby('Article')['Total_UVC'].idxmax(), ['Article', 'Ligne']]
default_line_df = default_line_df.reset_index(drop=True)

In [7]:
# 7. Create df_line_product_assignment
# Merge df_lines with article_line_uvc to get all line-product assignments
df_line_product_assignment = article_line_uvc.rename(columns={"Ligne": "Line code", "Article": "product code"})[["Line code", "product code"]]
# Merge with default_line_df to check if the line is the default for the product
df_line_product_assignment = df_line_product_assignment.merge(
    default_line_df.rename(columns={"Article": "product code", "Ligne": "Line code"}),
    on=["product code", "Line code"],
    how="left",
    indicator="is_default"
)
# Set is_default to True if the merge found a match, else False
df_line_product_assignment["is_default"] = df_line_product_assignment["is_default"] == "both"
df_line_product_assignment

,Line code,product code,is_default
0,PA02F05,300361,True
1,PA02F07,300361,False
2,PA02F05,300362,True
3,PA02F07,300362,False
4,PA03F01,300566,False
...,...,...,...
1442,PA06F12,601932,True
1443,PA02F07,601979,True
1444,PA02F03,601988,False
1445,PA02F06,601988,True


In [8]:
# 8. Create df_shift_configuration
df_shift_configuration = pd.DataFrame([
    {"name": "3x8 SS 5d", "description": "", "shifts_per_day": 3, "hours_per_shift": 8, "days_per_week": 5, "includes_saturday": True, "include_sunday": True},
    {"name": "2x8 5d", "description": "Two 8-hour shifts, weekdays only", "shifts_per_day": 2, "hours_per_shift": 8, "days_per_week": 5, "includes_saturday": False, "include_sunday": False},
    {"name": "1x12 6d", "description": "One 12-hour shift, Mon-Sat", "shifts_per_day": 1, "hours_per_shift": 12, "days_per_week": 6, "includes_saturday": True, "include_sunday": False},
    {"name": "3x8 7d", "description": "Three 8-hour shifts, all week", "shifts_per_day": 3, "hours_per_shift": 8, "days_per_week": 7, "includes_saturday": True, "include_sunday": True},
    {"name": "2x12 WE", "description": "Two 12-hour shifts, weekends only", "shifts_per_day": 2, "hours_per_shift": 12, "days_per_week": 2, "includes_saturday": True, "include_sunday": True}
])
df_shift_configuration

,name,description,shifts_per_day,hours_per_shift,days_per_week,includes_saturday,include_sunday
0,3x8 SS 5d,,3,8,5,True,True
1,2x8 5d,"Two 8-hour shifts, weekdays only",2,8,5,False,False
2,1x12 6d,"One 12-hour shift, Mon-Sat",1,12,6,True,False
3,3x8 7d,"Three 8-hour shifts, all week",3,8,7,True,True
4,2x12 WE,"Two 12-hour shifts, weekends only",2,12,2,True,True


In [9]:
# 9. Display all main DataFrames
print('df_lines:')
display(df_lines.head())
print('df_product:')
display(df_product.head())
print('df_sites:')
display(df_sites.head())
print('df_line_product_assignment:')
display(df_line_product_assignment.head())
print('df_shift_configuration:')
display(df_shift_configuration.head())

df_lines:


,Line code,Site name,site code
0,PA02F05,Dole,PA02
1,PA04F02,Hoerdt,PA04
2,PA02F01,Dole,PA02
3,PA02F02,Dole,PA02
4,PA02F06,Dole,PA02


df_product:


,product code,description,Type Produit,Type de recette,Type de matière,Type d'emballage
0,300361,CARREFOUR BRIS LOTX2 460G12CT SG,Roulée,brisée,MGV,Etui
1,300362,CARREFOUR FEUIL X2 460G 12CT SG,Roulée,feuilletée,MGV,Etui
2,300566,PINGO PT MASSA FOLHADA 230G 10CT MB,Roulée,feuilletée,MGV,FP_TRANSP
3,300567,PINGO PT MASSA QUEBRADA 230G 10CT MB,Roulée,brisée,MGV,FP_TRANSP
4,300802,CARREFOUR IT PASTA SFOGLIA 230G 10CT SG,Roulée,feuilletée,MGV,FPI_TOB


df_sites:


,Site,code_site
0,Dole,PA02
1,Hoerdt,PA04
2,Rivoli,PA05
3,SLB,PA06
4,Vittel,PA03


df_line_product_assignment:


,Line code,product code,is_default
0,PA02F05,300361,True
1,PA02F07,300361,False
2,PA02F05,300362,True
3,PA02F07,300362,False
4,PA03F01,300566,False


df_shift_configuration:


,name,description,shifts_per_day,hours_per_shift,days_per_week,includes_saturday,include_sunday
0,3x8 SS 5d,,3,8,5,True,True
1,2x8 5d,"Two 8-hour shifts, weekdays only",2,8,5,False,False
2,1x12 6d,"One 12-hour shift, Mon-Sat",1,12,6,True,False
3,3x8 7d,"Three 8-hour shifts, all week",3,8,7,True,True
4,2x12 WE,"Two 12-hour shifts, weekends only",2,12,2,True,True
